In [2]:
import sys
sys.path.append('../ry')
from libry import *
from numpy import *

In [3]:
# load and display a configuration
K = Config()
K.addFile('../test/boxProblem.g')
D = K.view()
K.getFrameNames()

['base',
 'box',
 'boxBo',
 'boxLe',
 'boxBa',
 'ballR',
 'block',
 'target',
 'boxBa>target']

In [4]:
# create a standard path optimization problem with 4 phases, each of .2 seconds, and 10 steps
komo = K.komo_path(5., 10, .2)

In [5]:
# create tau-DOFs (time intervals) in the optimization problem and add respective constraints
komo.addTimeOptimization()

In [6]:
# take one object, and first 'switch' it to become freely movable without costs,
# then, starting at phase .7, make it dynamic (imposing NE equations)
obj = 'ballR'
#komo.addSwitch_magic(.0, 'base', obj)
komo.makeObjectsFree(["ballR"]);

In [7]:
# (old) a way to constrain the object position to be 'before' the box before phase 0.2
#komo.addObjective(time=[0., .2], type=OT.ineq, feature=FS.position, frames=[obj], scaleTrans=[[0.,1.,0.]], target=[1.])

In [8]:
# objectives for collision (permanent) and target (at phase-time 4.)
komo.addObjective(type=OT.sos, feature=FS.accumulatedCollisions)
komo.addObjective(time=[5.], type=OT.eq, feature=FS.distance, frames=['target', obj], scale=[3e1])

In [9]:
# simple structure of this solution: bounce on the bottom of the box at times 1, 2, and 3
komo.addSwitch_dynamicTrans(1., -1., 'base', obj)
komo.addInteraction_elasticBounce(2., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(3., "boxBo", obj, .8)
komo.addInteraction_elasticBounce(4., "boxBo", obj, .8)

In [10]:
komo.optimize()

In [11]:
komoView = komo.view()

In [13]:
komo.display()

In [14]:
# read out solution: the full frame path, the tau path (time optimization), list of interaction forces
obj_path = komo.getPathFrames([obj])
tau_path = komo.getPathTau()
forces = komo.getForceInteractions()
forces

[{'at': 19,
  'force': [-4.015456525911921e-07,
   -1.7215543622597154e-07,
   -0.5171298129271731],
  'from': 'boxBo',
  'poa': [0.32476290545555636, -1.2552176094940752, 0.6500481299631874],
  'to': 'ballR'},
 {'at': 29,
  'force': [1.5467405073857602e-06,
   -6.143884595081588e-05,
   -0.3835327624544027],
  'from': 'boxBo',
  'poa': [0.2551594539321484, -0.36421934238798953, 0.6500108695741076],
  'to': 'ballR'},
 {'at': 39,
  'force': [4.259884974160733e-06,
   -3.042830037414587e-05,
   -0.3223542786732601],
  'from': 'boxBo',
  'poa': [0.18043970618711994, 0.4241143258233258, 0.6499953355226044],
  'to': 'ballR'}]

Conversion from steps to phase and time:
```
phase=0 is the FIXED initial configuration and corresponds to step=-1
step=0 is the first configuration s.t. optimization
\tau(s) gives the time between step s-1 and step s
phase(step s) = (s+1)/stepsPerPhase
time(step s) = sum i=0:s tau(i)
```

In [16]:
D=0
komoView=0
komo=0
K=0